## Build Alexnet Graph and load the pre-trained weights

In [2]:
%timeit
import pandas as pd
import cv2
from numpy import *
import tensorflow as tf
import glob
import numpy as np

def conv(input, kernel, biases, k_h, k_w, c_o, s_h, s_w, padding="VALID", group=1):
    '''From https://github.com/ethereon/caffe-tensorflow
        https://kratzert.github.io/2017/02/24/finetuning-alexnet-with-tensorflow.html
    '''
    c_i = input.get_shape()[-1]
    assert c_i % group == 0
    assert c_o % group == 0
    convolve = lambda i, k: tf.nn.conv2d(i, k, [1, s_h, s_w, 1], padding=padding)

    if group == 1:
        conv = convolve(input, kernel)
    else:
        input_groups = tf.split(input, group, 3)  # tf.split(3, group, input)
        kernel_groups = tf.split(kernel, group, 3)  # tf.split(3, group, kernel)
        output_groups = [convolve(i, k) for i, k in zip(input_groups, kernel_groups)]
        conv = tf.concat(output_groups, 3)  # tf.concat(3, output_groups)
    return tf.reshape(tf.nn.bias_add(conv, biases), [-1] + conv.get_shape().as_list()[1:])

In [4]:
%%time
train_x = zeros((1, 227,227,3)).astype(float32)
train_y = zeros((1, 1000))
xdim = train_x.shape[1:]
ydim = train_y.shape[1]

net_data = load(open("weights/bvlc_alexnet.npy", "rb"), encoding="latin1").item()

x = tf.placeholder(tf.float32, (None,) + xdim)


#conv1
#conv(11, 11, 96, 4, 4, padding='VALID', name='conv1')
k_h = 11; k_w = 11; c_o = 96; s_h = 4; s_w = 4
conv1W = tf.Variable(net_data["conv1"][0])
conv1b = tf.Variable(net_data["conv1"][1])
conv1_in = conv(x, conv1W, conv1b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=1)
conv1 = tf.nn.relu(conv1_in)

#lrn1
#lrn(2, 2e-05, 0.75, name='norm1')
radius = 2; alpha = 2e-05; beta = 0.75; bias = 1.0
lrn1 = tf.nn.local_response_normalization(conv1,
                                                  depth_radius=radius,
                                                  alpha=alpha,
                                                  beta=beta,
                                                  bias=bias)

#maxpool1
#max_pool(3, 3, 2, 2, padding='VALID', name='pool1')
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool1 = tf.nn.max_pool(lrn1, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)


#conv2
#conv(5, 5, 256, 1, 1, group=2, name='conv2')
k_h = 5; k_w = 5; c_o = 256; s_h = 1; s_w = 1; group = 2
conv2W = tf.Variable(net_data["conv2"][0])
conv2b = tf.Variable(net_data["conv2"][1])
conv2_in = conv(maxpool1, conv2W, conv2b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv2 = tf.nn.relu(conv2_in)


#lrn2
#lrn(2, 2e-05, 0.75, name='norm2')
radius = 2; alpha = 2e-05; beta = 0.75; bias = 1.0
lrn2 = tf.nn.local_response_normalization(conv2,
                                                  depth_radius=radius,
                                                  alpha=alpha,
                                                  beta=beta,
                                                  bias=bias)

#maxpool2
#max_pool(3, 3, 2, 2, padding='VALID', name='pool2')
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool2 = tf.nn.max_pool(lrn2, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)

#conv3
#conv(3, 3, 384, 1, 1, name='conv3')
k_h = 3; k_w = 3; c_o = 384; s_h = 1; s_w = 1; group = 1
conv3W = tf.Variable(net_data["conv3"][0])
conv3b = tf.Variable(net_data["conv3"][1])
conv3_in = conv(maxpool2, conv3W, conv3b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv3 = tf.nn.relu(conv3_in)

#conv4
#conv(3, 3, 384, 1, 1, group=2, name='conv4')
k_h = 3; k_w = 3; c_o = 384; s_h = 1; s_w = 1; group = 2
conv4W = tf.Variable(net_data["conv4"][0])
conv4b = tf.Variable(net_data["conv4"][1])
conv4_in = conv(conv3, conv4W, conv4b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv4 = tf.nn.relu(conv4_in)


#conv5
#conv(3, 3, 256, 1, 1, group=2, name='conv5')
k_h = 3; k_w = 3; c_o = 256; s_h = 1; s_w = 1; group = 2
conv5W = tf.Variable(net_data["conv5"][0])
conv5b = tf.Variable(net_data["conv5"][1])
conv5_in = conv(conv4, conv5W, conv5b, k_h, k_w, c_o, s_h, s_w, padding="SAME", group=group)
conv5 = tf.nn.relu(conv5_in)

#maxpool5
#max_pool(3, 3, 2, 2, padding='VALID', name='pool5')
k_h = 3; k_w = 3; s_h = 2; s_w = 2; padding = 'VALID'
maxpool5 = tf.nn.max_pool(conv5, ksize=[1, k_h, k_w, 1], strides=[1, s_h, s_w, 1], padding=padding)

#fc6
#fc(4096, name='fc6')
fc6W = tf.Variable(net_data["fc6"][0])
fc6b = tf.Variable(net_data["fc6"][1])
fc6 = tf.nn.relu_layer(tf.reshape(maxpool5, [-1, int(prod(maxpool5.get_shape()[1:]))]), fc6W, fc6b)

#fc7
#fc(4096, name='fc7')
fc7W = tf.Variable(net_data["fc7"][0])
fc7b = tf.Variable(net_data["fc7"][1])
fc7 = tf.nn.relu_layer(fc6, fc7W, fc7b)

#fc8
#fc(1000, relu=False, name='fc8')
fc8W = tf.Variable(net_data["fc8"][0])
fc8b = tf.Variable(net_data["fc8"][1])
fc8 = tf.nn.xw_plus_b(fc7, fc8W, fc8b)


#prob
#softmax(name='prob'))
prob = tf.nn.softmax(fc8)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

CPU times: user 9.63 s, sys: 10.2 s, total: 19.9 s
Wall time: 23.2 s


### Read the car_url_id to perform the extractions

In [5]:
df = pd.read_csv('balanced_cars.csv')

In [6]:
df.head(2)

,car_url_id,is_premium
0,2010-xuefolan-lechi-d77BsJBbW9uE,False
1,2009-fengtian-weichi-dIIJBBIhZZWZ,False


In [7]:
len(df)

3007

### Base on the car_url_id, read the images and augmented images
### And convert the images into extracted features and store in .txt / .txt_ format

In [8]:
%%time
#alexnet expected input is 227 * 227 *3
process_dimension = 227
for idx in range(len(df)):
#for idx in range(2):
 
    if idx % 50 == 0:
        print ('processing {}'.format(idx))

    car_url_id = df.loc[idx, 'car_url_id']
    image_file_name = '{}'.format(car_url_id)

    exist = glob.glob('extracted_alex/{}.txt'.format(car_url_id))
    if(len(exist)>0):
        print('this file already processed: {}, skip to next.'.format(image_file_name))
        continue
    
    try:
        #read the image file and resize it to 227 x 227
        img1 = cv2.imread('aug_img2/{}.png'.format(image_file_name))
        img2 = cv2.imread('aug_img2/{}_aug.png'.format(image_file_name))
        
        #print(img1.shape, img2.shape)
    except:
        print('img not found :', image_file_name, ' : ',img)
    fc7_ = sess.run(fc7, feed_dict={x: [img1]})
    np.savetxt('extracted_alex/{}.txt'.format(car_url_id), fc7_)
    
    fc7_ = sess.run(fc7, feed_dict={x: [img2]})
    np.savetxt('extracted_alex/{}_aug.txt'.format(car_url_id), fc7_)

processing 0
processing 50
processing 100
processing 150
processing 200
processing 250
processing 300
processing 350
processing 400
processing 450
processing 500
processing 550
processing 600
processing 650
processing 700
processing 750
processing 800
processing 850
processing 900
processing 950
processing 1000
processing 1050
processing 1100
processing 1150
processing 1200
processing 1250
processing 1300
processing 1350
processing 1400
processing 1450
processing 1500
processing 1550
processing 1600
processing 1650
processing 1700
processing 1750
processing 1800
processing 1850
processing 1900
processing 1950
processing 2000
processing 2050
processing 2100
processing 2150
processing 2200
processing 2250
processing 2300
processing 2350
processing 2400
processing 2450
processing 2500
processing 2550
processing 2600
processing 2650
processing 2700
processing 2750
processing 2800
processing 2850
processing 2900
processing 2950
processing 3000
CPU times: user 13min 39s, sys: 1min 26s, total